<a href="https://colab.research.google.com/github/dipayan10/Kolkata_dataset/blob/main/Clustering_in_kolkata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
pip install opencage

In [81]:
import numpy as np # library to handle data in a vectorized manner
 
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
 
import json # library to handle JSON files
 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
 
# import k-means from clustering stage
from sklearn.cluster import KMeans
 
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# libraries to read dataset from google drive
import requests 
from io import StringIO
 
print('Libraries imported.')

Libraries imported.


In [82]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [83]:
key = 'c0ca14c914034184a6792d41901f6a1e'
geocoder = OpenCageGeocode(key)

In [84]:
url = 'https://raw.githubusercontent.com/dipayan10/Kolkata_dataset/main/df_kolkata_final.csv'

In [85]:
df_kolkata = pd.read_csv(url)
df_kolkata.head()

,Ward No.,Councillors,Office Address
0,1,SMT. SITA JAISWARA,"9A, Gopi Mondal Lane, Kolkata-700002"
1,2,SMT. PUSPALI SINHA,"10, Rani Debendra Bala Rd Rishi, Paikpara Kolk..."
2,3,DR SANTANU SEN,"Seth Lane, Kolkata-50"
3,4,SHRI GOUTAM HALDAR,"9A, Raja Manindra Road, Kolkata-37"
4,5,SHRI TARUN SAHA,"67B, Khelat Babu Lane, Kolkata-37"


In [86]:
df_kolkata.shape

(144, 3)

In [87]:
df_kolkata.dtypes

Ward No.           int64
Councillors       object
Office Address    object
dtype: object

In [88]:
query = df_kolkata['Office Address'][0]
query

'9A, Gopi Mondal Lane, Kolkata-700002'

In [89]:
api_url = f'https://api.opencagedata.com/geocode/v1/json?q={query}&key=c0ca14c914034184a6792d41901f6a1e&language=en&pretty=1'
data = requests.get(api_url).json()
print(data['results'])



[{'annotations': {'DMS': {'lat': "22° 33' 45.46800'' N", 'lng': "88° 21' 46.94400'' E"}, 'MGRS': '45QXE4014195744', 'Maidenhead': 'NL42en35na', 'Mercator': {'x': 9836528.618, 'y': 2562822.943}, 'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=16/22.56263/88.36304&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=22.56263&mlon=88.36304#map=16/22.56263/88.36304'}, 'UN_M49': {'regions': {'ASIA': '142', 'IN': '356', 'SOUTHERN_ASIA': '034', 'WORLD': '001'}, 'statistical_groupings': ['LEDC']}, 'callingcode': 91, 'currency': {'alternate_symbols': ['Rs', '৳', '૱', '௹', 'रु', '₨'], 'decimal_mark': '.', 'html_entity': '&#x20b9;', 'iso_code': 'INR', 'iso_numeric': '356', 'name': 'Indian Rupee', 'smallest_denomination': 50, 'subunit': 'Paisa', 'subunit_to_unit': 100, 'symbol': '₹', 'symbol_first': 1, 'thousands_separator': ','}, 'flag': '🇮🇳', 'geohash': 'tunb6g2hb544sbp4kre4', 'qibla': 278.22, 'roadinfo': {'drive_on': 'left', 'speed_in': 'km/h'}, 'sun': {'rise': {'apparent': 16

In [90]:
demo_Latitude = data['results'][0]['geometry']['lat']
demo_Longitude = data['results'][0]['geometry']['lng']

In [ ]:
df_address = df_kolkata[['Office Address']]
df_address

In [92]:
my_columns = ['Office Address', 'Latitude', 'Longitude']
latlong_dataframe = pd.DataFrame(columns = my_columns)

In [93]:
latlong_dataframe.append(
       pd.Series(
    [
           df_kolkata['Office Address'][0],
           demo_Latitude,
           demo_Longitude
       ],
   index = my_columns
),
  ignore_index= True
)

,Office Address,Latitude,Longitude
0,"9A, Gopi Mondal Lane, Kolkata-700002",22.56263,88.36304


In [94]:
latlong_dataframe = pd.DataFrame(columns = my_columns)
for address in df_kolkata['Office Address']:
    api_url = f'https://api.opencagedata.com/geocode/v1/json?q={address}&key=c0ca14c914034184a6792d41901f6a1e&language=en&pretty=1'
    data = requests.get(api_url).json()
    latlong_dataframe = latlong_dataframe.append(
         pd.Series(
         [
             address,
             data['results'][0]['geometry']['lat'],
             data['results'][0]['geometry']['lng']
         ],
         index = my_columns),
    ignore_index = True
    )

In [ ]:
latlong_dataframe.head()

In [101]:
df_kolkata = pd.merge(df_kolkata, latlong_dataframe, on="Office Address")

In [102]:
df_kolkata.head(10)

,Ward No.,Councillors,Office Address,Latitude,Longitude
0,1,SMT. SITA JAISWARA,"9A, Gopi Mondal Lane, Kolkata-700002",22.562630,88.363040
1,2,SMT. PUSPALI SINHA,"10, Rani Debendra Bala Rd Rishi, Paikpara Kolk...",22.562630,88.363040
2,3,DR SANTANU SEN,"Seth Lane, Kolkata-50",22.586374,88.353731
3,4,SHRI GOUTAM HALDAR,"9A, Raja Manindra Road, Kolkata-37",22.612521,88.383454
4,5,SHRI TARUN SAHA,"67B, Khelat Babu Lane, Kolkata-37",22.562630,88.363040
5,6,SMT. SUMAN SINGH,"3/1B ,Turner Road, Kolkata-02",22.562630,88.363040
6,7,SHRI BAPI GHOSH,"47A, Bagbazar Street, Kolkata-700003",22.603919,88.366481
7,8,SHRI PARTHA MITRA,"3/1E Raja Raj Ballav Street, Kolkata-03",22.562630,88.363040
8,9,SMT. MITALI SAHA,"13A Madan Mohantala Street, Kolkata-05",22.562630,88.363040
9,10,SMT. KARUNA SENGUPTA,"Raja Nabakrishna Street, Kolkata-05",22.596719,88.366462


In [100]:
address = 'Kolkata, India'

geolocator = Nominatim(user_agent="kolkata_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kolkata are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kolkata are 22.5414185, 88.35769124388872.


In [104]:
# create map of New York using latitude and longitude values
map_kolkata = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, ward in zip(df_kolkata['Latitude'], df_kolkata['Longitude'], df_kolkata['Ward No.']):
    label = '{}'.format(ward)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kolkata)  
    
map_kolkata

In [122]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Ward', 
                  'Ward Latitude', 
                  'Ward Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [123]:
# type your answer here
kolkata_venues = getNearbyVenues(names=df_kolkata['Ward No.'],
                                   latitudes=df_kolkata['Latitude'],
                                   longitudes=df_kolkata['Longitude']
                                  )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144


In [124]:
kolkata_venues.shape

(722, 7)

In [125]:
kolkata_venues.head(10)

,Ward,Ward Latitude,Ward Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1,22.56263,88.36304,Prowess Pursuit Pvt Ltd,22.564407,88.363346,Business Service
1,1,22.56263,88.36304,Kathleen,22.561752,88.362460,Bakery
2,1,22.56263,88.36304,Georgian Inn,22.559555,88.361805,Hotel
3,1,22.56263,88.36304,Hind INOX,22.564914,88.359152,Multiplex
4,1,22.56263,88.36304,Santosh Mitra Square,22.566008,88.365719,Park
5,1,22.56263,88.36304,Entally Market,22.559952,88.366698,Market
6,2,22.56263,88.36304,Prowess Pursuit Pvt Ltd,22.564407,88.363346,Business Service
7,2,22.56263,88.36304,Kathleen,22.561752,88.362460,Bakery
8,2,22.56263,88.36304,Georgian Inn,22.559555,88.361805,Hotel
9,2,22.56263,88.36304,Hind INOX,22.564914,88.359152,Multiplex


In [126]:
kolkata_venues.groupby('Ward').count()

,Ward Latitude,Ward Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Ward,,,,,,
1,6,6,6,6,6,6
2,6,6,6,6,6,6
3,7,7,7,7,7,7
4,5,5,5,5,5,5
5,6,6,6,6,6,6
6,6,6,6,6,6,6
7,5,5,5,5,5,5
8,6,6,6,6,6,6
9,6,6,6,6,6,6


In [127]:
# one hot encoding
kolkata_onehot = pd.get_dummies(kolkata_venues[['Venue Category']], prefix="", prefix_sep="")

# add Ward column back to dataframe
kolkata_onehot['Ward'] = kolkata_venues['Ward']

# move Ward column to the first column
fixed_columns = [kolkata_onehot.columns[-1]] + list(kolkata_onehot.columns[:-1])
kolkata_onehot = kolkata_onehot[fixed_columns]

kolkata_onehot.head()

,Ward,ATM,American Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Awadhi Restaurant,Bakery,Bank,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Bus Station,Business Service,Café,Candy Store,Chinese Restaurant,Clothing Store,Convenience Store,Department Store,Dhaba,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store,Gym,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lounge,Market,Men's Store,Metro Station,Mobile Phone Shop,Movie Theater,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Nightclub,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Print Shop,Residential Building (Apartment / Condo),Restaurant,River,Shopping Mall,Snack Place,Spa,Supermarket,Tailor Shop,Tea Room,Theater,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Watch Shop,Women's Store
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
kolkata_grouped = kolkata_onehot.groupby('Ward').mean().reset_index()
kolkata_grouped

In [129]:
kolkata_grouped.shape

(135, 83)

In [ ]:
kolkata_grouped['Ward'] = kolkata_grouped['Ward'].astype(str)
kolkata_grouped.dtypes

In [138]:
num_top_venues = 5

for wards in kolkata_grouped['Ward']:
    print("----"+wards+"----")
    temp = kolkata_grouped[kolkata_grouped['Ward'] == wards].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----1----
       venue  freq
0  Multiplex  0.17
1     Market  0.17
2     Bakery  0.17
3       Park  0.17
4      Hotel  0.17


----2----
       venue  freq
0  Multiplex  0.17
1     Market  0.17
2     Bakery  0.17
3       Park  0.17
4      Hotel  0.17


----3----
                venue  freq
0  Light Rail Station  0.14
1   Indian Restaurant  0.14
2          Print Shop  0.14
3         Candy Store  0.14
4                Café  0.14


----4----
               venue  freq
0                ATM   0.2
1  Mobile Phone Shop   0.2
2               Park   0.2
3        Music Venue   0.2
4  Electronics Store   0.2


----5----
       venue  freq
0  Multiplex  0.17
1     Market  0.17
2     Bakery  0.17
3       Park  0.17
4      Hotel  0.17


----6----
       venue  freq
0  Multiplex  0.17
1     Market  0.17
2     Bakery  0.17
3       Park  0.17
4      Hotel  0.17


----7----
                venue  freq
0                Park   0.2
1  Chinese Restaurant   0.2
2                Bank   0.2
3               Rive

In [139]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Ward']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Wards_venues_sorted = pd.DataFrame(columns=columns)
Wards_venues_sorted['Ward'] = kolkata_grouped['Ward']

for ind in np.arange(kolkata_grouped.shape[0]):
    Wards_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kolkata_grouped.iloc[ind, :], num_top_venues)

Wards_venues_sorted.head()

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
1,2,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
2,3,Indian Restaurant,Candy Store,Light Rail Station,Café,Market,Snack Place,Print Shop,Women's Store,Grocery Store,Event Service
3,4,ATM,Music Venue,Electronics Store,Park,Mobile Phone Shop,Hotel,Hookah Bar,History Museum,Historic Site,Harbor / Marina
4,5,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant


###  Clustering Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [145]:
# set number of clusters
kclusters = 5

kolkata_grouped_clustering = kolkata_grouped.drop('Ward', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kolkata_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 3, 2, 3, 2, 0, 0, 0, 0, 0,
       0, 0, 4, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 3, 2, 3, 2, 0, 0, 0, 0,
       0, 2, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 3, 2,
       3, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 3, 3, 0, 1, 2, 2, 0, 2, 0,
       3, 0, 2, 2, 0, 3, 2, 1, 0, 2, 2, 3], dtype=int32)

Now we add the 'Cluster Labels' to the dataframe by creating a new column and merging it with the 'Ward_venues_sorted' and name the new dataframe 'kolkata_merged'.

In [151]:
Wards_venues_sorted.dtypes

Cluster Labels             int32
Ward                      object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object

In [146]:
# add clustering labels
Wards_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)




ValueError: ignored

In [154]:
kolkata_merged = df_kolkata
kolkata_merged.head(10)

,Ward No.,Councillors,Office Address,Latitude,Longitude
0,1,SMT. SITA JAISWARA,"9A, Gopi Mondal Lane, Kolkata-700002",22.562630,88.363040
1,2,SMT. PUSPALI SINHA,"10, Rani Debendra Bala Rd Rishi, Paikpara Kolk...",22.562630,88.363040
2,3,DR SANTANU SEN,"Seth Lane, Kolkata-50",22.586374,88.353731
3,4,SHRI GOUTAM HALDAR,"9A, Raja Manindra Road, Kolkata-37",22.612521,88.383454
4,5,SHRI TARUN SAHA,"67B, Khelat Babu Lane, Kolkata-37",22.562630,88.363040
5,6,SMT. SUMAN SINGH,"3/1B ,Turner Road, Kolkata-02",22.562630,88.363040
6,7,SHRI BAPI GHOSH,"47A, Bagbazar Street, Kolkata-700003",22.603919,88.366481
7,8,SHRI PARTHA MITRA,"3/1E Raja Raj Ballav Street, Kolkata-03",22.562630,88.363040
8,9,SMT. MITALI SAHA,"13A Madan Mohantala Street, Kolkata-05",22.562630,88.363040
9,10,SMT. KARUNA SENGUPTA,"Raja Nabakrishna Street, Kolkata-05",22.596719,88.366462


In [160]:
df_kolkata['Ward No.'] = df_kolkata['Ward No.'].astype(str)

In [161]:
df_kolkata.dtypes

Ward No.           object
Councillors        object
Office Address     object
Latitude          float64
Longitude         float64
dtype: object

In [162]:
kolkata_merged = df_kolkata

# merge kolkata_grouped with kolkata_data to add latitude/longitude for each neighborhood
kolkata_merged = kolkata_merged.join(Wards_venues_sorted.set_index('Ward'), on='Ward No.')


In [163]:
kolkata_merged = kolkata_merged.fillna(0)

In [164]:
kolkata_merged['Cluster Labels'] = kolkata_merged['Cluster Labels'].astype(int)

In [165]:
kolkata_merged.dtypes

Ward No.                   object
Councillors                object
Office Address             object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [166]:
kolkata_merged.head()

,Ward No.,Councillors,Office Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,SMT. SITA JAISWARA,"9A, Gopi Mondal Lane, Kolkata-700002",22.562630,88.363040,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
1,2,SMT. PUSPALI SINHA,"10, Rani Debendra Bala Rd Rishi, Paikpara Kolk...",22.562630,88.363040,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
2,3,DR SANTANU SEN,"Seth Lane, Kolkata-50",22.586374,88.353731,2,Indian Restaurant,Candy Store,Light Rail Station,Café,Market,Snack Place,Print Shop,Women's Store,Grocery Store,Event Service
3,4,SHRI GOUTAM HALDAR,"9A, Raja Manindra Road, Kolkata-37",22.612521,88.383454,2,ATM,Music Venue,Electronics Store,Park,Mobile Phone Shop,Hotel,Hookah Bar,History Museum,Historic Site,Harbor / Marina
4,5,SHRI TARUN SAHA,"67B, Khelat Babu Lane, Kolkata-37",22.562630,88.363040,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant


Now let's visualize the clusters 

In [167]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kolkata_merged['Latitude'], kolkata_merged['Longitude'], kolkata_merged['Ward No.'], kolkata_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


#### Cluster 1


We can see that that most of the neighborhoods fall under the 1st cluster. By examining the cluster, we see that the most common venues are coffee shops, pizzeria and restaurants.

In [168]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 0, kolkata_merged.columns[[1] + list(range(5, kolkata_merged.shape[1]))]]

,Councillors,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,SMT. SITA JAISWARA,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
1,SMT. PUSPALI SINHA,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
4,SHRI TARUN SAHA,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
5,SMT. SUMAN SINGH,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
7,SHRI PARTHA MITRA,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
8,SMT. MITALI SAHA,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
10,SHRI ATIN GHOSH,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
11,SMT. PRANATI BHATTACHARJEE,0,Park,Hotel,Bakery,Market,Business Service,Multiplex,Women's Store,Grocery Store,Event Service,Fast Food Restaurant
12,SHRI ANINDYA KISHOR ROUTH,0,0,0,0,0,0,0,0,0,0,0
16,SHRI MOHAN KUMAR GUPTA,0,0,0,0,0,0,0,0,0,0,0


#### Cluster 2


The 2nd cluster is populated with playgrouds, gyms, parks, yoga studios etc.

In [169]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 1, kolkata_merged.columns[[1] + list(range(5, kolkata_merged.shape[1]))]]

,Councillors,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,SMT. MAMATA MAJUMDAR,1,Train Station,Movie Theater,Women's Store,Harbor / Marina,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store
102,SMT. NANDITA ROY,1,Train Station,Women's Store,Harbor / Marina,Dhaba,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store


In [170]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 2, kolkata_merged.columns[[1] + list(range(5, kolkata_merged.shape[1]))]]

,Councillors,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,DR SANTANU SEN,2,Indian Restaurant,Candy Store,Light Rail Station,Café,Market,Snack Place,Print Shop,Women's Store,Grocery Store,Event Service
3,SHRI GOUTAM HALDAR,2,ATM,Music Venue,Electronics Store,Park,Mobile Phone Shop,Hotel,Hookah Bar,History Museum,Historic Site,Harbor / Marina
6,SHRI BAPI GHOSH,2,Park,River,Pier,Chinese Restaurant,Bank,Dhaba,Electronics Store,Event Service,Fast Food Restaurant,Flea Market
9,SMT. KARUNA SENGUPTA,2,Metro Station,Indian Sweet Shop,Fast Food Restaurant,Market,Bank,Historic Site,Event Service,Flea Market,Furniture / Home Store,Grocery Store
13,SHRI AMAL CHAKRABORTY,2,Historic Site,Clothing Store,Gym,Harbor / Marina,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store
15,SHRI SADHAN SAHA,2,Vegetarian / Vegan Restaurant,Indian Restaurant,Plaza,Bakery,Women's Store,Gym,Electronics Store,Event Service,Fast Food Restaurant,Flea Market
18,SMT. SIKHA SAHA,2,Indian Restaurant,IT Services,Women's Store,Harbor / Marina,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store
27,IQBAL AHMED,2,Women's Store,Theater,Hotel,Department Store,Ice Cream Shop,Grocery Store,Dhaba,Electronics Store,Event Service,Fast Food Restaurant
32,SHRI PABITRA BISWAS,2,ATM,Theater,Restaurant,Gym,Dhaba,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store
33,SMT. ALOANANDA DAS,2,Theater,Restaurant,Women's Store,Gym,Dhaba,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store


In [171]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 3, kolkata_merged.columns[[1] + list(range(5, kolkata_merged.shape[1]))]]

,Councillors,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,SMT. SHUKLA BHORE,3,Historic Site,Café,Bakery,Business Service,Harbor / Marina,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store
17,SHRI SUNANDA SARKAR,3,Café,Bakery,Performing Arts Venue,Ice Cream Shop,Nightclub,Hotel,Tea Room,Pizza Place,Restaurant,Flea Market
38,MD. JASIMUDDIN,3,Café,Bakery,Performing Arts Venue,Ice Cream Shop,Nightclub,Hotel,Tea Room,Pizza Place,Restaurant,Flea Market
40,SMT. REITA CHOWDHURY,3,Bookstore,Hotel,Café,Metro Station,Park,Art Museum,Ice Cream Shop,Gym,Event Service,Fast Food Restaurant
49,SMT. MOUSUMI DEY,3,Department Store,Park,Café,Bakery,Business Service,Gym,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store
68,SHRI SUKDEV CHAKRABARTY,3,Café,Chinese Restaurant,Ice Cream Shop,Women's Store,Harbor / Marina,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store
71,SHRI SANDIP RANJAN BAKSHI,3,Café,Hotel,Market,Pharmacy,Grocery Store,Women's Store,Gym,Electronics Store,Event Service,Fast Food Restaurant
85,SMT. TISHTA BISWAS (DAS),3,Café,Indian Restaurant,Ice Cream Shop,Lounge,Clothing Store,Hotel,Chinese Restaurant,Bengali Restaurant,Women's Store,Grocery Store
86,SHRI SUBRATA GHOSH,3,Shopping Mall,Café,Multiplex,Market,Movie Theater,Gym,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store
94,SHRI TAPAN DASGUPTA,3,Photography Studio,Café,Park,Mobile Phone Shop,IT Services,Furniture / Home Store,Indian Restaurant,Electronics Store,Event Service,Ice Cream Shop


In [172]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 4, kolkata_merged.columns[[1] + list(range(5, kolkata_merged.shape[1]))]]

,Councillors,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,SMT. MINAKSHI GUPTA,4,History Museum,Women's Store,Harbor / Marina,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store,Gym
121,SMT. SOMA CHAKRABORTY,4,History Museum,Art Museum,Historic Site,Electronics Store,Event Service,Fast Food Restaurant,Flea Market,Furniture / Home Store,Grocery Store,Gym
